In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

pd.set_option('display.max_columns', None)

In [ ]:
api_key = "431e779057msh1f0c87c2937a329p125efcjsn45064e4b4925" # Define la clave de la API

In [ ]:
class RecolectorPeliculas:  # Define una clase llamada RecolectorPeliculas
    def __init__(self, api_key):  # Define un método inicializador para la clase
        self.base_url = "https://moviesdatabase.p.rapidapi.com"  # Asigna la URL base de la API
        self.headers = {  # Define un diccionario con los encabezados de la solicitud
            "X-RapidAPI-Key": api_key,  # Clave de la API para autenticación
            "X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"  # Host de la API 
        }

    def llamar_pelis_api(self, year1, year2, genre):  # Define un método para llamar a la API de películas
        url = f"{self.base_url}/titles"  # Define la URL completa para la solicitud a la API
        datos_paginas = []  # Inicializa una lista vacía para almacenar los datos
        querystring = {"genre": genre, "startYear": year1, "titleType": "movie", "endYear": year2}  # Parámetros de la consulta

        while url:  # Inicia un bucle while que se ejecuta mientras la URL no esté vacía
            if not url.startswith('http'):  # Verifica si la URL no comienza con 'http'
                url = self.base_url + url  # Añade la URL base si es necesario
            response = requests.get(url, headers=self.headers, params=querystring)  # Realiza una solicitud GET a la API

            if response.status_code == 200:  # Comprueba si la solicitud fue exitosa
                json_pelis = response.json()  # Convierte la respuesta en JSON
                datos_paginas.extend(json_pelis['results'])  # Agrega los resultados a la lista de páginas de datos
                url = json_pelis.get('next')  # Actualiza la URL para iterar por las páginas de resultados
                querystring = {}  # Limpia los parámetros de consulta para la próxima solicitud
            else:  # Si la solicitud falla
                print("Error en la solicitud:", response.status_code, response.reason)  # Imprime un mensaje de error
                return None  # Retorna None debido a un error

        return {"results": datos_paginas}  # Retorna un diccionario con los datos recopilados

    def limpiar_json(datos_paginas):  # Define un método para limpiar los datos JSON obtenidos
        pelis = []  # Crea una lista vacía para almacenar los datos de películas
        for movie in datos_paginas['results']:  # Itera sobre cada película en los resultados
            type = movie['titleType']['text']  # Extrae el tipo de título de la película
            name = movie['titleText']['text']  # Extrae el nombre de la película
            year = movie['releaseYear']['year']  # Extrae el año de lanzamiento de la película
            month = movie['releaseDate']['month'] if 'releaseDate' in movie and movie['releaseDate'] is not None else 'No consta'  # Extrae el mes de lanzamiento o asigna 'No consta'
            id_movie = movie['id']  # Extrae el ID de la película
            
            pelis.append({  # Añade los datos de la película al diccionario de películas
                'Tipo': type,  # Tipo de película
                'Nombre': name,  # Nombre de la película
                'Año de Estreno': year,  # Año de estreno de la película
                'Mes de Estreno': month,  # Mes de estreno de la película
                'ID': id_movie  # ID de la película
            })
        return pd.DataFrame(pelis)  # Retorna un DataFrame de pandas con los datos de películas

    def conseguir_datos_pelis(self, year1, year2, genre):  # Define un método para obtener los datos de las películas
        datos = self.llamar_pelis_api(year1, year2, genre)  # Llama al método para llamar a la API de películas
        if datos:  # Si se obtienen datos correctamente
            return self.limpiar_json(datos)  # Llama al método para limpiar los datos y retorna el DataFrame resultante
        else:  # Si falla la obtención de datos
            return None  # Retorna None

def convertir_csv(df, nombrecsv):  # Define una función para convertir un DataFrame a CSV
    df.to_csv(nombrecsv, index=False)  # Convierte el DataFrame a CSV y lo guarda en el archivo especificado

In [ ]:
# Ejemplo de uso
coleccion_pelis = RecolectorPeliculas(api_key)  # Crea una instancia de la clase RecolectorPeliculas con la clave API proporcionada

# Obtener datos de películas de un género específico entre 2010 y 2024
df_pelis = coleccion_pelis.conseguir_datos_pelis(2010, 2024, "Drama")  # Llama al método para obtener los datos de películas de comedia
print(df_pelis.head(50))  # Imprime los primeros 50 registros del DataFrame de películas obtenido

In [ ]:
# Guardar los datos en un archivo CSV
convertir_csv(df_pelis, "Películas_Comedia_2010-2024.csv")  # Llama a la función para convertir y guardar los datos en un archivo CSV